### Домашнее задание № 5
* Цель:
    Разработать классификатор для определения спам/не спам сообщений с использованием различных методов предобработки текста и векторизации.
* Задания:
    1. Подготовка данных:
        - Загрузите датасет SMS сообщений, размеченных как спам или не спам. https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
    2. Предобработка текста:
        - Реализуйте лемматизацию или стемминг для текстов сообщений.
    3. Векторизация текста:
        - Примените мешок слов (Bag of Words) и TF-IDF для векторизации текста.
    4. Моделирование:
        - Постройте модели для классификации сообщений как спам или не спам (модели на ваш выбор).
        - Сравните результаты моделей с использованием различных методов векторизации и предобработки текста.
    5. Сравнительный анализ:
        - Оцените качество моделей с различными комбинациями предобрабо

In [10]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('spam.csv', encoding='latin-1')

# Инспекция данных
print(data.head())
print(data.info())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
m

In [11]:
data = data[['v1', 'v2']]
# Переименовываем столбцы
data.columns = ['label', 'message']

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Загрузка необходимых ресурсов
nltk.download('stopwords')
nltk.download('wordnet')

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = text.split()
    # Лемматизация и удаление стоп-слов
    text = ' '.join(lemmatizer.lemmatize(word) for word in text if word not in stop_words)
    return text

# Применение функции предобработки к сообщениями
data['processed_message'] = data['message'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Женя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Женя\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Bag of Words
count_vectorizer = CountVectorizer()
X_bow = count_vectorizer.fit_transform(data['processed_message'])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data['processed_message'])

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Разделение данных
# Преобразуем метки в числовые значения
y = data['label'].map({'ham': 0, 'spam': 1})

# Обучение и оценка моделей для Bag of Words
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=42)

clf_lr_bow = LogisticRegression()
clf_lr_bow.fit(X_train_bow, y_train)
y_pred_lr_bow = clf_lr_bow.predict(X_test_bow)
print("Logistic Regression (Bag of Words)")
print(classification_report(y_test, y_pred_lr_bow))

clf_nb_bow = MultinomialNB()
clf_nb_bow.fit(X_train_bow, y_train)
y_pred_nb_bow = clf_nb_bow.predict(X_test_bow)
print("Naive Bayes (Bag of Words)")
print(classification_report(y_test, y_pred_nb_bow))

clf_rf_bow = RandomForestClassifier()
clf_rf_bow.fit(X_train_bow, y_train)
y_pred_rf_bow = clf_rf_bow.predict(X_test_bow)
print("Random Forest (Bag of Words)")
print(classification_report(y_test, y_pred_rf_bow))

# Обучение и оценка моделей для TF-IDF
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

clf_lr_tfidf = LogisticRegression()
clf_lr_tfidf.fit(X_train_tfidf, y_train)
y_pred_lr_tfidf = clf_lr_tfidf.predict(X_test_tfidf)
print("Logistic Regression (TF-IDF)")
print(classification_report(y_test, y_pred_lr_tfidf))

clf_nb_tfidf = MultinomialNB()
clf_nb_tfidf.fit(X_train_tfidf, y_train)
y_pred_nb_tfidf = clf_nb_tfidf.predict(X_test_tfidf)
print("Naive Bayes (TF-IDF)")
print(classification_report(y_test, y_pred_nb_tfidf))

clf_rf_tfidf = RandomForestClassifier()
clf_rf_tfidf.fit(X_train_tfidf, y_train)
y_pred_rf_tfidf = clf_rf_tfidf.predict(X_test_tfidf)
print("Random Forest (TF-IDF)")
print(classification_report(y_test, y_pred_rf_tfidf))

Logistic Regression (Bag of Words)
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       1.00      0.85      0.92       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Naive Bayes (Bag of Words)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.91      0.93      0.92       150

    accuracy                           0.98      1115
   macro avg       0.95      0.96      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Random Forest (Bag of Words)
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.83      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92    